#  Instruction-Tuning with LLMs


Instruction-based fine-tuning, referred to as instruction LLAMA. It trains the language models to follow specific instructions and generate appropriate responses. For instruction-tuning, the dataset plays an important role as it provides structured examples of instructions, contexts, and responses, allowing the model to learn how to handle various tasks effectively. Instruction LLAMA often uses human feedback to refine and improve model performance; however, this lab doesn't cover this aspect.

The context and instruction are concatenated to form a single input sequence that the model can understand and use to generate the correct response.

#### Context and instruction

	•	Instruction: A command to specify what the model should do
	•	Context: Additional information or background required for performing the instruction
	•	Combined input: The instruction and context combine together into a single input sequence

In [ ]:
!pip install torch torchvision torchaudio

In [14]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, pipeline
from datasets import load_dataset
from peft import AutoPeftModelForCausalLM, LoraConfig, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer

# Load a tokenizer and format function

In [2]:
template_tokenizer = AutoTokenizer.from_pretrained(
    "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
)

def format_prompt(example):
    """Format the prompt to using the <|user|> template TinyLLama
    is using
    """

    # Format answers
    chat = example["messages"]
    prompt = template_tokenizer.apply_chat_template(chat, tokenize=False)

    return {"text": prompt}


### Load and format the data using the template TinyLLama is using

In [3]:
dataset = (
  load_dataset("HuggingFaceH4/ultrachat_200k", split="test_sft")
    .shuffle(seed=42)
    .select(range(3_000))
)

dataset = dataset.map(format_prompt)

Example of formatted prompt

In [4]:
print(dataset["text"][2576])

<|user|>
Given the text: Knock, knock. Who’s there? Hike.
Can you continue the joke based on the given text material "Knock, knock. Who’s there? Hike"?</s>
<|assistant|>
Sure! Knock, knock. Who's there? Hike. Hike who? Hike up your pants, it's cold outside!</s>
<|user|>
Can you tell me another knock-knock joke based on the same text material "Knock, knock. Who's there? Hike"?</s>
<|assistant|>
Of course! Knock, knock. Who's there? Hike. Hike who? Hike your way over here and let's go for a walk!</s>



# Model Quantization

Now that we have our data, we can start loading in our model. This is where
we apply the Q in QLoRA, namely quantization. We use the
`bitsandbytes` package to compress the pretrained model to a 4-bit
representation.
In BitsAndBytesConfig, you can define the quantization scheme. We
follow the steps used in the original QLoRA paper and load the model in 4-
bit (`load_in_4bit`) with a normalized float representation
(`bnb_4bit_quant_type`) and double quantization
(`bnb_4bit_use_double_quant`):

In [5]:
model_name = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"

# 4-bit quantization configuration - The Q in QLoRA
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, # Use 4-bit precision model loading
    bnb_4bit_quant_type="nf4", # Quantization type
    bnb_4bit_compute_dtype="float16", # Compute dtype
    bnb_4bit_use_double_quant=True, # Apply nested quantization
)

### Load the model to train on the GPU

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",

    # Leave this out for regular SFT
    quantization_config=bnb_config
)
model.config.use_cache = False
model.config.pretraining_tp = 1

### Load LLaMA tokenizer

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = "<PAD>"
tokenizer.padding_side = "left"

We use `padding_side = "left"` with TinyLlama (decoder-only model) to ensure that useful tokens are at the end of the sequence. This is important because these models generate text autoregressively and expect informative tokens to come last. With left padding, the attention mask correctly ignores `<PAD>` tokens on the left.

This quantization procedure allows us to decrease the size of the original
model while retaining most of the original weights’ precision. Loading the
model now only uses ~1 GB VRAM compared to the ~4 GB of VRAM it
would need without quantization. 

> Note that during fine-tuning, more
VRAM will be necessary so it does not cap out on the ~1 GB VRAM
needed to load the model.

# LoRA Configuration
Next, we will need to define our LoRA configuration using the peft
library, which represents hyperparameters of the fine-tuning process:

In [8]:
# Prepare LoRA Configuration
peft_config = LoraConfig(
    lora_alpha=32,  # LoRA Scaling
    lora_dropout=0.1,  # Dropout for LoRA Layers
    r=64,  # Rank
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[ 
        "k_proj",
        "gate_proj",
        "v_proj",
        "up_proj",
        "q_proj",
        "o_proj",
        "down_proj",
    ], # Layers to target 
)

# Prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

`r`
This is the rank of the compressed matrices. 
Increasing this value will also increase the sizes of compressed
matrices leading to less compression and thereby improved
representative power. Values typically range between 4 and 64.

`lora_alpha`
Controls the amount of change that is added to the original weights. In
essence, it balances the knowledge of the original model with that of the
new task. A rule of thumb is to choose a value twice the size of `r`.

`target_modules`
Controls which layers to target. The LoRA procedure can choose to
ignore specific layers, like specific projection layers. This can speed up
training but reduce performance and vice versa.

# Training Configuration 

In [9]:
OUTPUT_DIR = "./results"

# Training Arguments
training_arguments = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    num_train_epochs=1,
    logging_steps=10,
    fp16=True,
    gradient_checkpointing=True
)

# Training

In [10]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    max_seq_length=512,

    # Leave this out for regular SFT
    peft_config=peft_config,
)

# Train model
trainer.train()

# Save QLoRA weights
QLORA_MODEL = "TinyLlama-1.1B-qlora"
trainer.model.save_pretrained(QLORA_MODEL)

/home/pedro/anaconda3/envs/torch-env/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/pedro/.local/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/pedro/.local/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/pedro/.local/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:408: UserWarning: You passed a tokenizer with `padding_side`

{'loss': 1.6705, 'grad_norm': 0.26160547137260437, 'learning_rate': 0.00019964928592495045, 'epoch': 0.03}


  5%|▌         | 20/375 [01:13<21:30,  3.64s/it]

{'loss': 1.4756, 'grad_norm': 0.25835326313972473, 'learning_rate': 0.0001985996037070505, 'epoch': 0.05}


  8%|▊         | 30/375 [01:49<21:11,  3.69s/it]

...

{'loss': 1.4441, 'grad_norm': 0.19025741517543793, 'learning_rate': 7.885298685522235e-07, 'epoch': 0.96}


 99%|█████████▊| 370/375 [22:27<00:18,  3.71s/it]

{'loss': 1.4522, 'grad_norm': 0.19409486651420593, 'learning_rate': 8.771699011416168e-08, 'epoch': 0.99}


100%|██████████| 375/375 [22:47<00:00,  3.65s/it]


{'train_runtime': 1367.461, 'train_samples_per_second': 2.194, 'train_steps_per_second': 0.274, 'train_loss': 1.416849353790283, 'epoch': 1.0}


# Merge Weights

After we have trained our QLoRA weights, we still need to combine them
with the original weights to use them. We reload the model in 16 bits,
instead of the quantized 4 bits, to merge the weights. Although the tokenizer
was not updated during training, we save it to the same folder as the model
for easier access:

In [13]:
model = AutoPeftModelForCausalLM.from_pretrained(
    "TinyLlama-1.1B-qlora", # QLORA_MODEL
    low_cpu_mem_usage=True,
    device_map="auto",
)

# Merge LoRA and base model
merged_model = model.merge_and_unload()

After merging the adapter with the base model, we can use it with the
prompt template that we defined earlier:

In [15]:
# Use our predefined prompt template
prompt = """<|user|>
Tell me something about Large Language Models.</s>
<|assistant|>
"""

# Run our instruction-tuned model
pipe = pipeline(task="text-generation", model=merged_model,
tokenizer=tokenizer)
print(pipe(prompt)[0]["generated_text"])

<|user|>
Tell me something about Large Language Models.</s>
<|assistant|>
Large Language Models (LLMs) are a type of artificial intelligence (AI) that can generate human-like language. They are trained on large amounts of data, including text, audio, and video, and are capable of generating complex and nuanced language.

LLMs are used in a variety of applications, including natural language processing (NLP), machine translation, and chatbots. They can be used to generate text, speech, or images, and can be trained to understand different languages and dialects.

One of the most significant applications of LLMs is in the field of natural language generation (NLG). LLMs can be used to generate text in a variety of languages, including English, French, and German. They can also be used to generate speech, such as in a chatbot or voice assistant.

LLMs have the potential to revolutionize the way we communicate and interact with each other. They can help us create more engaging and personal